In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
# print((purchase_data_df.dtypes))  #Satisfactory

## Player Count

* Display the total number of players


In [3]:
purchase_data_df.head(20)
# purchase_data.describe()
# SN (screen name) will provide identifier for unique persons playing, and number of players.

# purchase_data_df["SN"].describe(), shows 576 unique
player_count = len(purchase_data_df["SN"].unique())
print(f" Total number of players = {player_count}")

 Total number of players = 576


## Purchasing Analysis (Total)

In [4]:
#Quick analysis of data frame
purchase_data_df.head(20)

#Retrieve data, and map to format
unique_items = len(purchase_data_df["Item Name"].unique())

av_purch_price = purchase_data_df["Price"].mean()
# no_purch = (purchase_data_df["Purchase ID"]).tail() # shows 779 + 1st purchase = 780 purchases
no_purch = len(purchase_data_df["Purchase ID"])
rev = purchase_data_df["Price"].sum()

#Create new dataframe for summary data to be presented 
purch_analysis_df = pd.DataFrame({"Number of Unique Items": [unique_items],
                                  "Average Price": [av_purch_price],
                                  "Number of Purchases":[no_purch],
                                  "Total Revenue": [rev]})

#Format each column using dict mapping
purch_analysis_df["Average Price"] = purch_analysis_df["Average Price"].map("${:,.2f}".format)
purch_analysis_df["Total Revenue"] = purch_analysis_df["Total Revenue"].map("${:,.2f}".format)

purch_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [5]:
# purchase_data_df.head()
# count_gender = purchase_data_df["Gender"].value_counts()
# count_gender
# gender_summary = pd.DataFrame({"G"})

# Need SN and Gender columns, to obtain list of unique players and their gender
reduced_df = purchase_data_df[["SN","Gender","Purchase ID"]]
#Drop duplicates
reduced_df_SN = reduced_df.value_counts(subset=['SN','Gender'])     #Retuns 576 unqiue players, duplicates subtracted
gender_group = reduced_df_SN.groupby(["Gender"])
gender_compare = gender_group.count()
#print(type(gender_compare))     #Output is series

#Convert count series output into dataframe, set headers titles, sort values ascending order
final_compare_df = gender_group.count().rename_axis('Gender').reset_index(name='Total Count').sort_values(['Total Count'], ascending=False)
# final_compare_df2 = final_compare_df[["Gender","Total Count"]]  #Don't know how to drop index
final_compare_df3 = final_compare_df.set_index("Gender")

#Percentage calcs
#Sum population & create variable for total, calc every row against sum and create new column, format fro %2f
total = final_compare_df3["Total Count"].sum()
Percent = final_compare_df3["Total Count"]/total*100
final_compare_df3["Percentage of Players"] = Percent
final_compare_df3.style.format({"Percentage of Players": "{:.2f}%"})

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [6]:
#Method, 
# (1) groupby on raw data, 3 times for 3 different functions (ie. count, mean, sum)
# (2) convert subsequent series to df (to_frame()), rename columns where appropriate
# (3) merge dfs and format

#:::::: (1) & (2)::::::
# Retrieve purchase count by gender
gender_count = purchase_data_df.groupby(["Gender"])["Gender"].count()                       # Groupby gender, sum on price
gender_count_df = gender_count.to_frame()                                                   # Convert series > df
gender_count_df_rn = gender_count_df.rename(columns={"Gender": "Purchase Count"})           # Rename columns

#Retrieve total purchases by gender - calc before "Ave purch price" b/c need gender_purch for calcs later
gender_purch = purchase_data_df.groupby(["Gender"])["Price"].sum()                          
gender_purch_df = gender_purch.to_frame()                                                   
gender_purch_df_rn = gender_purch_df.rename(columns={"Price": "Total Purchase Value"})      

#Retrieve average purchase price by gender
gender_avg = round((gender_purch / gender_count), 2)
gender_avg_df = gender_avg.to_frame()
gender_avg_df_rn = gender_avg_df.rename(columns={0: "Average Purchase Price"})

#Retrieve purchase per person by gender. Using gender_compare (which gives a count, no duplicates) variable from "Gender Demographics"
per_person_gender = round((gender_purch / gender_compare), 2)
per_person_gender_df = per_person_gender.to_frame()
per_person_gender_df_rn = per_person_gender_df.rename(columns={0: "Avg Total Purchase per Person"})



#:::::: (3)::::::
merge_df = pd.merge(gender_count_df_rn, gender_avg_df_rn, on="Gender", how="outer")
merge_df123 = pd.merge(merge_df, gender_purch_df_rn, on="Gender", how="outer")
merge_df1234 = pd.merge(merge_df123, per_person_gender_df_rn, on="Gender", how="outer")

# Format
merge_df1234["Average Purchase Price"] = merge_df1234["Average Purchase Price"].map("${:,.2f}".format)
merge_df1234["Total Purchase Value"] = merge_df1234["Total Purchase Value"].map("${:,.2f}".format)
merge_df1234["Avg Total Purchase per Person"] = merge_df1234["Avg Total Purchase per Person"].map("${:,.2f}".format)
merge_df1234

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [7]:
# purchase_data_df["Age"].describe(), although duplicates exist, max 45 min 7
# # Creating bins
bins = [0, 9.9, 13.99, 19.9, 24.9, 29.9, 34.9, 39.9, 49.9]

# Create labels for the bins
group_labels = ["<10", "10-14", "15-19","20-24","25-29","30-34","35-39", "40+"]


reduced_df = purchase_data_df[["SN","Age"]]
reduced_df

# reduced_df.reset_index()

# Drop duplicate SNs
dup_out = reduced_df.drop_duplicates(subset=['SN','Age'], keep="first")   #Returns 576, corresponds to prior calcs, correct
dup_out.dtypes

#Categorise players
dup_out["Ages"] = pd.cut(dup_out["Age"], bins, labels= group_labels)
dup_out.head()

#Create a GroupBy object based upon "Age"
dup_out_group = dup_out.groupby("Ages")

# Count how many fall within each bin by using count and print series
dup_out_group["SN"].count()

# Convert series back into df using reset_index
age_demo_df = dup_out_group["SN"].count().reset_index()
age_demo_df

# Calc % of players and format .00%
age_demo_df["Percentage of Players"] = age_demo_df["SN"] / age_demo_df["SN"].sum()
age_demo_df["Percentage of Players"] = age_demo_df["Percentage of Players"].astype(float).map("{:.2%}".format)

# Rename SN to Total Count
age_demo_df = age_demo_df.rename(columns={"SN": "Total Count"})
final_age_demo_df = age_demo_df.set_index("Ages")
final_age_demo_df


c:\Users\Steve\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Total Count,Percentage of Players
Ages,,
<10,17,2.95%
10-14,20,3.47%
15-19,109,18.92%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [10]:
# Groupby Age Ranges (present in raw data)
age_range_count = purchase_data_df.groupby(["Age Ranges"])["Purchase ID"].count()               # count no. of purchases per age range, on purchase id.
age_range_count_df = age_range_count.to_frame()
age_range_count_df_rn = age_range_count_df.rename(columns={"Purchase ID": "Purchase Count"})
age_range_count_df_rn

# Retrieve total purchases per age group
total_purch_value = purchase_data_df.groupby(["Age Ranges"])["Price"].sum()
total_purch_value_df = total_purch_value.to_frame()
total_purch_value_df_rn = total_purch_value_df.rename(columns={"Price": "Total Purchase Value"})

# Retrieve average purchase price by age group
av_purch_price_age = round((total_purch_value / age_range_count), 2)
av_purch_price_age_df = av_purch_price_age.to_frame()
av_purch_price_age_df_rn = av_purch_price_age_df.rename(columns={0: "Average Purchase Price"})

# Retrieve unique players by age group, to be used as denominator in calculating 'avg total purchase p/person'
unique_age = purchase_data_df.groupby(["Age Ranges"])["SN"].nunique()

# Calc 'avg total purchase p/person', 
av_tot_purch_age = round(total_purch_value / unique_age, 2)
av_tot_purch_age_df = av_tot_purch_age.to_frame()
av_tot_purch_age_df_rn = av_tot_purch_age_df.rename(columns={0: "Avg Total Purchase per Person"})


# Merge DFs
merge_purch_analysis_age12 = pd.merge(age_range_count_df_rn, av_purch_price_age_df_rn, on="Age Ranges", how="outer")
merge_purch_analysis_age123 = pd.merge(merge_purch_analysis_age12, total_purch_value_df_rn, on="Age Ranges", how="outer")
merge_purch_analysis_age1234 = pd.merge(merge_purch_analysis_age123, av_tot_purch_age_df_rn, on="Age Ranges", how="outer")

# Format
merge_purch_analysis_age1234["Average Purchase Price"] = merge_purch_analysis_age1234["Average Purchase Price"].map("${:,.2f}".format)
merge_purch_analysis_age1234["Total Purchase Value"] = merge_purch_analysis_age1234["Total Purchase Value"].map("${:,.2f}".format)
merge_purch_analysis_age1234["Avg Total Purchase per Person"] = merge_purch_analysis_age1234["Avg Total Purchase per Person"].map("${:,.2f}".format)
merge_purch_analysis_age1234

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,26,$2.92,$75.87,$3.79
15-19,138,$3.04,$419.80,$3.85
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [70]:
# Groupby individuals, and sum on purchase price, to determine total spend per individual
top_spenders_total = purchase_data_df.groupby(["SN"])["Price"].sum() 
top_spenders_total_df = top_spenders_total.to_frame()
top_spenders_total_df_rn = top_spenders_total_df.rename(columns={"Price": "Total Purchase Value"})

# Retrieve number of individual purchases per individual
top_spenders_count = purchase_data_df.groupby(["SN"])["Purchase ID"].count() 
top_spenders_count_df = top_spenders_count.to_frame()
top_spenders_count_df_rn = top_spenders_count_df.rename(columns={"Purchase ID": "Purchase Count"})

# Calc avg purchase price, with the data obtained above
av_purch_price_ind = round((top_spenders_total / top_spenders_count), 2)
av_purch_price_ind_df = av_purch_price_ind.to_frame()
av_purch_price_ind_df_rn = av_purch_price_ind_df.rename(columns={0: "Average Purchase Price"})


# Merge DFs
top_spenders_merge12 = pd.merge(top_spenders_count_df_rn, av_purch_price_ind_df_rn, on="SN", how="outer")
top_spenders_merge123 = pd.merge(top_spenders_merge12, top_spenders_total_df_rn, on="SN", how="outer")


# Sort DF descending on purchase value
top_spenders_merge123_sorted = top_spenders_merge123.sort_values(by=["Total Purchase Value"], ascending=False)


# Format
top_spenders_merge123_sorted["Average Purchase Price"] = top_spenders_merge123_sorted["Average Purchase Price"].map("${:,.2f}".format)
top_spenders_merge123_sorted["Total Purchase Value"] = top_spenders_merge123_sorted["Total Purchase Value"].map("${:,.2f}".format)
top_spenders_merge123_sorted.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [67]:
# Create new DF with required columns/fields
mostpop_df = purchase_data_df[["Item ID", "Item Name", "Price"]]

# Groupby Item ID and Item Name, and count on item id, returning number of times item sold
mostpop_df_idname_count = mostpop_df.groupby(["Item ID", "Item Name"])["Item ID"].count().to_frame()
mostpop_df_idname_count_rn = mostpop_df_idname_count.rename(columns={"Item ID": "Purchase Count"})


# Groupby Item ID and Item Name, and median on price, which will return the unit price of the item.
mostpop_df_idname_price = mostpop_df.groupby(["Item ID", "Item Name"])["Price"].median().to_frame()
mostpop_df_idname_price_rn = mostpop_df_idname_price.rename(columns={"Price": "Item Price"})


# Groupby Item ID and Item Name, and sum on price, which will return total purchase value of this item sold.
mostpop_df_idname_value = mostpop_df.groupby(["Item ID", "Item Name"])["Price"].sum().to_frame()
mostpop_df_idname_value_rn = mostpop_df_idname_value.rename(columns={"Price": "Total Purchase Value"})


# Merge 
mostpop_merge12 = pd.merge(mostpop_df_idname_count_rn, mostpop_df_idname_price_rn, on=['Item ID','Item Name'])
mostpop_merge123 = pd.merge(mostpop_merge12, mostpop_df_idname_value_rn, on=['Item ID','Item Name'])


# Sort DF descending on purchase count, to find popular. 
# Need to sort before formatting, b/c when convert to $59, that becomes a string, not float
mostpop_merge123_sorted_pop = mostpop_merge123.sort_values("Purchase Count", ascending = False)


# Format
mostpop_merge123_sorted_pop["Item Price"] = mostpop_merge123_sorted_pop["Item Price"].map("${:,.2f}".format)
mostpop_merge123_sorted_pop["Total Purchase Value"] = mostpop_merge123_sorted_pop["Total Purchase Value"].map("${:,.2f}".format)


mostpop_merge123_sorted_pop.head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
75,Brutality Ivory Warmace,8,$2.42,$19.36
103,Singed Scalpel,8,$4.35,$34.80
34,Retribution Axe,8,$2.22,$17.76


## Most Profitable Items

In [66]:
# Sort DF descending on purchase count, to find popular. 
# Need to sort before formatting, b/c when convert to $59, that becomes a string, not float
mostpop_merge123_sorted_value = mostpop_merge123.sort_values("Total Purchase Value", ascending = False)


# Format
mostpop_merge123_sorted_value["Item Price"] = mostpop_merge123_sorted_value["Item Price"].map("${:,.2f}".format)
mostpop_merge123_sorted_value["Total Purchase Value"] = mostpop_merge123_sorted_value["Total Purchase Value"].map("${:,.2f}".format)

mostpop_merge123_sorted_value.head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16
